In [1]:
AUTHOR_NAME = 'Jane Deijnen'
AUTHOR_ID_NR = '1354396'
AUTHOR_DATE = '2020-05-05'

AUTHOR_NAME, AUTHOR_ID_NR, AUTHOR_DATE

('Jane Deijnen', '1354396', '2020-05-05')

In [13]:
import numpy as np
import pandas as pd
import bokeh as bk
import random

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import seaborn as sns
sns.set()  # set Seaborn defaults
plt.rcParams['figure.figsize'] = 10, 5  # default hor./vert. size of plots, in inches
plt.rcParams['lines.markeredgewidth'] = 1  # to fix issue with seaborn box plots; needed after import seaborn
from sklearn.cluster import KMeans  # for clustering

from bokeh.io import output_notebook, show, reset_output, curdoc
from bokeh.models import Slider, CustomJS
from bokeh.plotting import figure
from bokeh.layouts import layout, column
from bokeh.models import (
    ColumnDataSource, Div,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
    FactorRange,
    ImageURL
)
from bokeh.palettes import BuPu
from bokeh.palettes import Colorblind8
output_notebook()

Loading BokehJS ...

In [3]:
#import eye tracking data
eyetracking_data = pd.read_csv('metro_data.csv', encoding = 'latin1', sep = ";")
eyetracking_data.head()

,Timestamp,StimuliName,FixationIndex,FixationDuration,MappedFixationPointX,MappedFixationPointY,user,description
0,2586,01_Antwerpen_S1.jpg,9,250,1151,458,p1,color
1,2836,01_Antwerpen_S1.jpg,10,150,1371,316,p1,color
2,2986,01_Antwerpen_S1.jpg,11,283,1342,287,p1,color
3,3269,01_Antwerpen_S1.jpg,12,433,762,303,p1,color
4,3702,01_Antwerpen_S1.jpg,13,183,624,297,p1,color


In [4]:
#dataframe of Antwerper_s1 stimulus only
Antwerpen_s1 = eyetracking_data[eyetracking_data['StimuliName'] == '01_Antwerpen_S1.jpg']
Antwerpen_s1.head()

,Timestamp,StimuliName,FixationIndex,FixationDuration,MappedFixationPointX,MappedFixationPointY,user,description
0,2586,01_Antwerpen_S1.jpg,9,250,1151,458,p1,color
1,2836,01_Antwerpen_S1.jpg,10,150,1371,316,p1,color
2,2986,01_Antwerpen_S1.jpg,11,283,1342,287,p1,color
3,3269,01_Antwerpen_S1.jpg,12,433,762,303,p1,color
4,3702,01_Antwerpen_S1.jpg,13,183,624,297,p1,color


In [5]:
#color palette
colors = Colorblind8

In [6]:
#cluster slider
slider_cluster = Slider(title="Amount of Clusters", start=1, end=8, value=2, step=1)

In [7]:
#create source
source = ColumnDataSource(data=dict(x=[], y=[], c=[], u=[], color=[]))

In [34]:
#draw empty figure
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"
TOOLTIPS=[("Participant", "@u"),
          ("AOI", "@c"),
         ("X-coordinate", "@x"),
         ("Y-coordinate", "@y")
         ]

p = figure(title="AOI plot", tools=TOOLS, tooltips=TOOLTIPS)
p.image_url(url=['01_Antwerpen_S1.jpg'], x=0, y=0, w=1650, h=1200) #not sure yet how to determine the width and hight
p.circle(x='x', y='y', color='color', legend_field='c', source=source, fill_alpha=0.2, size=10)
p.xgrid.grid_line_color = None
p.legend.orientation = "vertical"
p.legend.location = "bottom_right"
p.legend.title = 'AOI'
p.y_range.flipped = True

show(p)

In [30]:
#function calculating new dataframe based on slider value
def select_graphs():
    X_km = Antwerpen_s1[['MappedFixationPointX', 'MappedFixationPointY']].copy()
    km = KMeans(slider_cluster.value)
    km.fit(X_km)
    centers = pd.DataFrame(km.cluster_centers_, columns=X_km.columns)
    X_km['cluster'] = km.labels_

    user = Antwerpen_s1["user"]
    X_km = X_km.join(user)
    X_km = X_km.reset_index()
    
    X_km_adj = X_km.copy()

    for i in range(X_km.index[-1]+1):
        X_km_adj.loc[i, 'cluster'] = X_km_adj['cluster'][i]+1

    return X_km_adj

In [31]:
#function updating column data source base on slider value
def update():
    df = select_graphs()
    x = list(df['MappedFixationPointX'])
    y = list(df['MappedFixationPointY'])
    c = list(df['cluster'])
    u = list(df['user'])
    
    colorlist = []
    for i in range(df.index[-1]+1):
        index = df['cluster'][i]
        colorlist.append(colors[index])
    
    color = colorlist
   
    source.data = dict(
        x=x,
        y=y,
        c=c,
        u=u,
        color=color
    )

In [32]:
#layout execution
controls = [slider_cluster]
for control in controls:
    control.on_change('value', lambda attr, old, new: update())

inputs = column(*controls, width=320, height=1000)
inputs.sizing_mode = "fixed"

l = layout([
    [inputs, p],
], sizing_mode="fixed")

update()  # initial load of the data

curdoc().add_root(l)
curdoc().title = "AOI plot"

In [33]:
show(l)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

